# SVMen - Comp 598 - Machine Learning - Project 2 Report

---
### David Rapoport, Ethan Macdonald, Charlie Bloomfield

---
### 1. Introduction
In this report, we discuss our methods for classifying NPR interviews from their text content into one of four categories: Author Interviews, Movie Interviews, Music Interviews, and Interviews. Given an annotated dataset of 54,000 interviews, we implement three vectorizing techniques: Term Frequency–Inverse Document Frequency (TF-IDF), Bag of Words (BoW), and *n*-grams. We then explore modelling with a Support Vector Classifier (SVC) provided by scikit-learn, Naive Bayes (NB) implemented by our team, and *k*-Nearest Neighbor (KNN) implemented by our team. Using standard cross validation techniques, we identify Naive Bayes as the best classifier. We then focus on tuning our feature selection and hyperparameter combinations to improve the classification performance of this model.

---
### 2. Running the Code

In order to run our code, we execute main.py. When the program asks for a configuration file, we may enter a custom configuration file or simply leave the input as blank to use a default configuration file. A configuration file defines which vectorizors, feature selectors, and learners we wish to use.

After we select a configuration file, the program displays a list of possible combinations to choose from. Each combination consists of a vectorizor, selector, and learner. To select every item, simply type 'a'. Once we have selected the combinations we wish to use, we enter the path to the data file we wish to use. To use the default path type 'd'.

In [ ]:
execfile("main.py")

Enter the name of the config file in python import notation.
 File must be in config directory. [Default config.default]


The program then trains a model for each of the selected combinations and outputs the accuracy of each model. At the end, the program returns information about the best model based on accuracy (TODO: validation set accuracy? test set accuracy? cross validation?). This information includes which vectorizor, selector, learner and parameters were used.

---
### 3. Data Preprocessing
The data is originally given as a CSV file containing id,text,label. Before creating different feature sets we first load the text and targets and perform preprocesssing on the text. Our preprocessing consists of several steps in which we normalize the data. First the text is tokenized using the nltk wordpunct tokenizer. After that we transform each token into lowercase, removing all non alphabetic characters (parentheses, commas, apostrophes, etc.), and remove words of length less than 3. This reason for this last step is that we want to remove the tail ends of contractions as well as any words such as "be", "is", "am" which are length two and are most likely irrelevant stop words. An example of the pipeline is shown below. After this we use the NLTK WordNetLemmatizer to lemmatize the word. We chose to lemmatize rather than stem because lemmatization is more likely to return a proper english word. This means we could later leverage the meaning of the word (for example with word2vec). The final step of the preprocessing was the remove stop words. The stop words used were the nltk stopwords with "__EOS__" appended to the list of stopwords. [NLTK CITATION HERE]

In [7]:
from nltk.tokenize import wordpunct_tokenize
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

print(wordpunct_tokenize("Shouldn't can't isn't doesn't!!!"))
tokens = wordpunct_tokenize("Shouldn't this author sing? He isn't Michael Jackson!!!")
print([lemmatizer.lemmatize(word.lower()) for word in tokens if len(word)>2 and word.isalpha()])

['Shouldn', "'", 't', 'can', "'", 't', 'isn', "'", 't', 'doesn', "'", 't', '!!!']
['shouldn', 'this', 'author', 'sing', 'isn', 'michael', 'jackson']


---
### 4. Feature Design and Selection

For this project, we used scikit_learn's CountVectorizer and TfidfVectorizer as well as our home grown Word2VecVectorizer for feature design. We used scikit_learn's SelectPercentile for feature selection. 

#### 4.1 Investigating the data

Before beginning with feature design and selection it is useful to examine the train data we are given. We found that there were 54803 unigrams with 17722 of those unigrams only appearing in the corpus once. The top word was "know" appearing 62000 times, the next were "like", "think", "one", "well", "people", and "really" with frequencies rangings from 37000 - 23000. These words are very likely to be stopwords and future improvements may add these words to the stopword list or tweak with the __max_df__ parameter to remove them. 

Next, we found the 100 most frequent words from each class. We intersected these sets to get a list of 70 words which appear in the top 100 for all four classes. Again, in the future these could be appended to the list of stopwords for a possibly better classifier (A quick test revealed that doing this improved the score of our best classified by 0.003). Finally we performed a set subtraction of set(top_100_for_class_x) - set(70_common_to_all_classes) and sorted based on frequency in the original corpus. The results are shown below

| Class | 1st most frequent | 2nd | 3rd |4th | 5th |
| --- | --- |---| --- | --- | --- |
| Author | book | talking | woman | american | npr |
| Movie | film | movie | play | show | talking|
| Music | song | music | play | show | npr |
| Interview | book | show | talking | woman | american |

Class 4 seems to have a great deal of overlap with the other classes. Investigation of the learners will show that this class was the hardest to classify. Another interesting future analysis of the data might be to look at set(top_100_for_class_x) - Union(set(top_100_for_class_j)) where j!=x. Nonetheless, the current method of analysis manages to remove 70/100 from each class. 

The final step in the investigation was to look at some of the interviews. We found that our thought process in trying to classify the interviews relied heavily on looking for the key words which we associate with an author, movie, or music. We also found that our decision rule in classifying documents which were in class four was NOT(Class1) and NOT(Class2) and NOT(Class3). We attempt to use this observation later to build our classifiers. The full list of frequent words can be seen in the appendix.

#### 4.2 Word2Vec

One of the vectorizers we used leveraged the word2vec library. Once trained, you can use word2vec to project words into $ R^n $. In our case, we used a pretrained google model where $n=300$. Once we have a vector representation of the word we can use it to compute distance between other words[Word2Vec CITATION HERE]. We decided to subclass the CountVectorizer class from scikit_learn. Our fit method would first call CountVectorizer.fit() to convert from raw text into a matrix representation in $R^{nxm}$. We then create an $m$ dimensional vector x as follows: 

$$x_i=max(word2vec.distance(word_i, keyword) \forall keyword \in Keywords)$$

The keywords used were simply the class labels "author", "movie", "music". We then select either the top k features or the top k percent features based on $x$. The value of k and which method of selection were hyperparameters to the Vectorizer. The choice of keywords was made due to the difficulty discerning class four which was noted in the previous section. We believed that a by only selecting features related to the three classes which were labeled we could learn the fourth class as simply a negative class. The results are shown below 

|Word|Distance|Keyword|Word Frequency|
|---|---|---|---|
|"film"| 0.867 | "movie"| 7748 |
|"coauthor"| 0.729| "author"| 10 |
|"jazz"| 0.683 | "music" | 1223 |
|"sequel" | 0.65777931269028578 | "movie" | 66 |

It is clear that there is an issue with using the above criteria to select features. The issue is that "coauthor" will be selected as a feature, despite the fact it only appears 10 times in the corpus. Another idea we had was to assign values in x equal to $distance\times \log frequency $. This would allow us to select as features words which were not only relevant to the classes but also frequent enough that they would appear in many examples and the test set. The list sorted with respect to this new metric appears below 

|Word|Distance|Keyword|Word Frequency|
|---|---|---|---|
|"book"| 0.540 | "author" | 11526|
|"song" | 0.543 | "music" | 10476|
| "jazz"| 0.683 | "music" | 1223 |
| "writer"| 0.586 | "author" | 1995|
| "musical" | 0.618 | "music" | 1021 | 



#### 4.3 Results

Despite our high hopes with Word2Vec, the python library gensim which we were using does not provide a way to perform vectorized comparisons of distance. This meant that the Word2VecVectorizer was incredibly slow and impractical for use. We also found that while using the first metric to select features, we were doing worse than with a plain count vectorizer. Some of that was surely to do with the lack of consideration for word frequency, but due to time constraints the $distance\times \log frequency $ vectorizer was never used. 

We also found that scikit_learns tfidf vectorizer performed quite poorly. Comparing TfidfVectorizer and CountVectorizer we would score between 0.05-0.10 worse using TfidfVectorizer. Through cross validation we found that the optimal parameters were to consider unigrams and bigrams, all words provided that they appear more than once in the document, and to not binarize the data. After this we performed a chi squared test on the features and kept the top fifty percent. Future works include implementing Interact [CITATION] and Binormal Seperation [CITATION]


---
### 5. Parameter Selection
In order to select optimal hyperparameters for our models, we employ GridSearchCV from scikit-learn, which exhaustively searches a manually defined range of hyperparameters. This search is guided by performance on a our validation set.

TODO: Figures — comparison of various hyperparameters?

---
### 6. Testing and Validation
We set up our mini testing and validation framework to train and test with single 80/20 train/test splits on a fraction of the entire dataset. Typically, when trying to improve hyperparameters or optimizations, we train on a randomly selected sample of 1% of the dataset and the single train/test split. While this gives us a lower test classification accuracy, it lets us  isolate which of our feature set, model, and hyper parameter selections give the best results quickly. When exploring new models with many hyperparameters, the ability to have a development enviroment that allows us to iterate quickly proves critical to our success in finetuning our model hyperparameters and feature selection techniques. This is especially the case for models with polynomial runtimes, such as SVC, where the difference in runtime between 1% and 100% of the data is a several orders of magnitude improvement.

---
### 7. Algorithm Selection

##### 7.1 Naive Bayes

We implemented several different varients of the NaiveBayes Classifier. The first three are the Bernoulli, Multinomial, and Continuous (Gaussian) NaiveBayes. Another two came from Rennie, Shih, Teevan, and Karger [CITATION] and are called Complementary Naive Bayes, and Weight Normalized Complementary NaiveBayes. Finally we had a failed attempt at what we call "Ignore Class Four" Naive Bayes.

All of the classifiers first learn a prior. The prior for class y is calculated by $\frac{N(Y)}{\sum_Y N(Y)}$ Where $N(y)$ is the number of documents labeled with class $y\in Y$. 

##### Classic Naive Bayes

Next for the Bernoulli Naive Bayes we learn $Pr(x_j|Y=y)$ by computing 
$$\frac{ \sum_{d\in y} x_j + l }{N(Y=y) +\alpha\times l} $$
where $\alpha $ and $l$ are hyper parameters. Prediction is then done by choosing the class which is most likely given the data. To find $Pr(Y=y|x)$ we compute 

$$\log(Pr(Y=y)) + x \times \log (Pr(x|y)) + (1-x) \times \log (1-Pr(x|y))$$

However, in practice subtracting a constant from a sparse matrix does not work so instead we do the following

$$ \log(Pr(Y=y)) + x \times (\log(Pr(x |y ) - \log(1-Pr(x|y))) + \log(1-Pr(x|y))$$

as is done in the scikit_learn documentation.


The Multinomial Naive Bayes learns $Pr(x_j|Y=y)$ by computing 

$$ \log( \sum_{d\in y} x_j + \alpha ) - \log( N(y) + \alpha\times m) $$

And to predict we take ArgMax $\log(Pr(Y=y)) + x \times Pr(x|y)$ Note that we already took the log when we trained our classifier so we do not need to take the log of $Pr(x|y)$ for prediction.

Finally, for continuous NaiveBayes for $Pr(x|y)$ we learn the mean and standard deviation for each feature for each class and then use those along with the normal pdf to predict.

##### From the literature

We also implemented Complementary Naive Bayes as per the literature. This is exactly like multinomial NaiveBayes except instead of learning $Pr(x|y)$ using the data in class y, we use the data not in class y. This helps overcome any possible class skew in the multiclass situation. After that, to predict we simply take the argmax of $P(y) - x*P(x|y)$. The key difference here being that we subtract from the prior since we are trying to minimize unlikelihood. The other Naive Bayes we implemented was Weight Normalized Complementary Naive Bayes. This is exactly like Complementary Naive Bayes except after training, for each class y every weight $x_j$ is converted to $\frac{x_j}{\sum_y |x_j|}$.

##### Ignore class four Naive Bayes

As we observed in section 4.1, it is very difficult to determine whether an article is in class 4. Our intuitions led us to believe it would be best to learn a classifier that is NOT(class1) and NOT(class2) and NOT(class3). We proceeded to implement the following Naive Bayes. Remove all instances of class 4 from your train set. Learn a normal Complementary Naive Bayes on the remaining three classes. Now using all of the class four interviews do the following $Threshold = \frac{\sum_{x\in y_4}Max(P(y=1|x), P(y=2|x), P(y=3|x))}{N(y=4)}$. Now to predict we just find the argmax as usual, if the argmax is below the learned threshold, output 4. Unfortunately, this learned performed quite poorly.

##### Results

We found that the best performance was with Complementary Naive Bayes with alpha=1.0. Below we show the confusion matrix of the learner. We see that as we thought, it only scored 45% on class 4 problems.
!ConfusionMatrix(cnb_confusion.png)


##### 7.2 *k*-Nearest Neighbor
We implement K Nearest Neighbor as our standard algorithm with flexible support for parameterized kernel functions. To do so, we allow for clients of the KNearestNeighbor class to define a function that takes as parameters two feature vectors of equal lengths and return a scalar representing the distance between the two. We provide two kernel functions in 'classifiers.kernels': euclidian distance and cosine distance. By default, the classifier uses the cosine distance kernel as this is appropriate in the domain of text classification.

While our initial unit tests show that KNN works for generated datasets, we discover errors in implementing it properly with sparse vectors. The size of the provided dataset requires a sparse vector representation for in memory processing. We consider rewriting the implementation, but tests with scikit learn's KNN model is performing poorly and we choose to focus our time on optimizing SVM and NB.


##### 7.3 SVM
Finally, we test using scikit learn's Support Vector Classifier to classify the NPR articles. This implementation provides many model hyperparameters and optimization parameters. We explore two of these parameters: the penalty value for misclassifications and the kernel used by the SVC.

Initial attempts to run SVC with the default parameters on the entire dataset proves to be infeasible on our machines due to runtime constraints. The polynomial runtime of SVC leads to a several hour runtime with no response. So we start again and try to assess it's classification percentage on 1% of the dataset. With the previous framework for validating on a single 80/20 train/test split, we get relatively impressive initial results with default parameters. Coupled with the optimum count vect feature set, we see a ~58% classification rate.


![figures/figure_2.png]


With these initial results, we explore optimizing the misclassifcation penalty and kernel parameters to the SVC. We are within %10 range from our best Naive Bayes classification results and hope the optimizations might make the difference. To do so, we perform a search over the set of all scikit learn's provided kernels [rbf", "linear", "poly", "sigmoid", "precomputed"] with each of the C values [0.1, 1.0, 2.5, 5.0]. While we are able to iterate over all of the provided kernels, we arbitrarily choose four values for C. Testing over a range of values for C is optimal but grows combinatorially with the number of inputs and quickly becomes too expensive to execute. For example, it would have been ideal to iterate over many float values of C between 0 and 1, but this is computationally unrealistic. With the values shown above, we identify the "linear" coupled with a C value of 2.5 as the best combination for SVC classification.


With the insight we gained from running SVC on 1% of the data, we are ready to train on all the data and try make a submission to see how our training generalizes to test data. We are still running into problems with running SVC on the entire dataset in a reasonable amout of time, so we choose to run our code on a remote server provided by [Cloud Digital](https://www.digitalocean.com/) with improved processing power. We install and run our program to train the SVC model overnight, but an entire night of execution on the remote server proves to be insufficient to train the model on the entire dataset and we terminate it's execution.

Unable to complete model training on the entire dataset, we consider using SVC by training it on a fraction of the dataset and using that model as a learner for submission. But any such learning shows classification percentages lower then our best with Naive Bayes, so we do not use it as a classifier for submissions.


---
### 9. Further Discussion
After final test submissions, we dropped from 6th on the leaderboards to 18th. This is a significant drop - the second most of any team. Our final test submission classified 60.764% correctly, down from > 62%. This suggests we either overtrained to the test data or did not cross validate sufficiently. Given our reliance on a single 80/20 train test split in developing, we find it likely that our validation techniques did not accurately represent the true performance of our models. We made a total of 4 submissions, each in the range of 60% - 63%. While three of the four submissions were improvements over the previous, the low number of submissions gave us little feedback on the variance in our models on generalizing to test data. Because we did not run k-fold validation on the training data, we relied instead on the values that we saw on separate program invocations. While we did notice model variance, we did not quanitify it with cross validation. As such there's no way to validate our submissions were in the range that we anticipated.


---
### 10. Statement of Faith
We hereby state that all the work presented in this report is that of the authors.

---
### 11. References

TODO: scikit-learn reference

TODO: David's refs from facebook?